<a href="https://colab.research.google.com/github/mfmarlonferrari/tcc2020recommender/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#https://pypi.org/project/mysql-connector/
!pip install mysql-connector flask-ngrok

In [7]:
import pandas as pd
import mysql.connector

Conexão com a base de Dados transacional

In [148]:
host="database-1.c6nlhkjxpqnn.sa-east-1.rds.amazonaws.com"
port=3306
dbname="DB_TCC"
user="admin"
password="admin123"

conn = mysql.connector.connect(user=user, password=password,
                              host=host,
                              database=dbname)

cur = conn.cursor()

cur.execute('select * from questions')
dados = cur.fetchall()
dataset = pd.DataFrame(dados, columns=['ID', 'USUARIO', 'LIVRO', 'TITULO', 'RATING','D1','D2'])[['USUARIO','LIVRO','TITULO','RATING']]
dataset.head(5)

,USUARIO,LIVRO,TITULO,RATING
0,1,4071158,O Menino do Pijama Listrado,5
1,2,2595026,O Pequeno Príncipe,5
2,2,87263,O Alienista,5
3,4,3484606,Harry Potter e a Pedra Filosofal (Harry Potter...,5
4,4,72196,"Harry Potter e a Câmara Secreta (Harry Potter,...",5


In [149]:
dataset.shape

(38, 4)

In [154]:
movies_features = dataset.pivot_table(index='USUARIO', columns='TITULO', values='RATING').fillna(0)
movies_features.head()

TITULO,"A Batalha do Labirinto (Percy Jackson e os Olimpianos, #4)",A Cabana,"A Casa de Hades (Heróis do Olimpo, #4)","A Maldição do Titã (Percy Jackson e os Olimpianos, #3)","A marca de Atena (Os heróis do Olimpo, #3)",Data Science para negócios: O que você precisa saber sobre mineração de dados e pensamento analítico de dados,Desculpe a Nossa Falha,Dom Casmurro,"Harry Potter and the Sorcerer's Stone (Harry Potter, #1)","Harry Potter e a Câmara Secreta (Harry Potter, #2)","Harry Potter e a Ordem da Fênix (Harry Potter, #5)","Harry Potter e a Pedra Filosofal (Harry Potter, #1)","Harry Potter e as Relíquias da Morte (Harry Potter, #7)","Harry Potter e o Cálice de Fogo (Harry Potter, #4)","Harry Potter e o Enigma do Príncipe (Harry Potter, #6)","Harry Potter e o Prisioneiro de Azkaban (Harry Potter, #3)",O Alienista,O Alquimista,"O Arqueiro (A Busca do Graal, #1)","O Filho de Netuno (Heróis do Olimpo, #2)",O Imbecil Coletivo I - Atualidades Inculturais Brasileiras,"O Ladrão de Raios (Percy Jackson e os Olimpianos, #1)","O Mar de Monstros (Percy Jackson e os Olimpianos, #2)",O Menino Maluquinho,O Menino do Pijama Listrado,O Pequeno Príncipe,"O Sangue do Olimpo (Os Heróis do Olimpo, #5)","O Último Adeus De Sherlock Holmes (Sherlock Holmes, #4)","O Último Olimpiano (Percy Jackson e os Olimpianos, #5)",Poderosa (Poderosa #1),"The Adventures of Sherlock Holmes (Sherlock Holmes, #3)","The Lost Hero (The Heroes of Olympus, #1)",Viagem ao Centro da Terra
USUARIO,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,4.0,5.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,5.0,5.0,5.0,5.0,0.0,4.0,3.0,5.0,0.0,5.0,5.0,0.0,5.0,0.0,5.0,0.0,5.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
4,0.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [155]:
from sklearn.neighbors import NearestNeighbors

modelo = NearestNeighbors(metric='cosine', n_neighbors=3)
modelo.fit(movies_features)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                 radius=1.0)

In [156]:
id_usuario = 4

distancia, indices_vizinhos = modelo.kneighbors(movies_features.loc[id_usuario].values.reshape(1,-1), n_neighbors=4)

In [157]:
for i in range(0, len(distancia.flatten())):
  if i==0:
    print('Recomendando para o usuário {0}:\n'.format(id_usuario))
  elif movies_features.index[indices_vizinhos.flatten()[i]] != id_usuario:
    print('Recomendado vizinho: {0} com distância de {1}'.format(movies_features.index[indices_vizinhos.flatten()[i]], distancia.flatten()[i]))

Recomendando para o usuário 4:

Recomendado vizinho: 3 com distância de 0.7123633432461501
Recomendado vizinho: 1 com distância de 0.7785441652280276
Recomendado vizinho: 5 com distância de 1.0


In [158]:
id_recomendado = movies_features.index[indices_vizinhos.flatten()][1]

In [159]:
base = movies_features.loc[id_usuario].to_frame()
recomendado = movies_features.loc[id_recomendado].to_frame()

In [160]:
novos_titulos = pd.merge(base,recomendado,on='TITULO',how='inner')
novos_titulos = novos_titulos[(novos_titulos[id_recomendado] > 0) & (novos_titulos[id_usuario] == 0)].reset_index()
novos_titulos

,TITULO,4,3
0,Data Science para negócios: O que você precisa...,0.0,5.0


In [1]:
!pip install flask-ngrok

In [12]:
from flask_ngrok import run_with_ngrok
from flask import Flask
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/")
def recomendar(idUsuario):
    return "<h1>Running Flask on Google Colab!</h1>"
  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://f4cc4f26e7a1.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [25/Oct/2020 17:14:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2020 17:14:16] "GET /favicon.ico HTTP/1.1" 404 -
